In [1]:
import numpy as np
from PIL import Image
import io
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy import stats
import hashlib
import warnings
import random
import string
warnings.filterwarnings('ignore')
# pyright: ignore[reportMissingImports]

# ====================== LOGISTIC MAP ======================
def generate_logistic_sequence(x0, r, length, discard=2000):
    """
    Generate Logistic Map chaotic sequence
    Formula: x_{n+1} = r * x_n * (1 - x_n)
    Domain: x0 in (0, 1), r in [0.0, 4.0]
    """
    sequence = np.zeros(length + discard)
    x = float(x0)
    
    for i in range(length + discard):
        # Logistic Map formula
        x = r * x * (1.0 - x)
        sequence[i] = x
    
    # Discard transient
    return sequence[discard:]

def generate_keystream(seq, img_shape):
    """Generate keystream from chaotic sequence."""
    height, width, channels = img_shape
    total_pixels = height * width * channels
    
    if len(seq) < total_pixels:
        seq = np.tile(seq, (total_pixels // len(seq) + 1))
        seq = seq[:total_pixels]
    
    keystream = ((seq * 10000) % 256).astype(np.uint8)
    return keystream.reshape(height, width, channels)

# ====================== KEY GENERATION ======================
def text_to_parameters(key_text, r=None):
    """
    Convert text key to chaotic parameters using SHA-256 hash.
    If r is provided, use it; otherwise, generate from key.
    """
    # Generate SHA-256 hash of the key
    key_hash = hashlib.sha256(key_text.encode()).hexdigest()
    
    # Convert first 8 characters to x0 (0.1 to 0.9)
    x0_part = int(key_hash[:8], 16)
    x0 = 0.1 + (x0_part % 8000) / 10000  # x0 in [0.1, 0.9]
    
    # If r is not provided, generate from key
    if r is None:
        r_part = int(key_hash[8:16], 16)
        r = 3.57 + (r_part % 4300) / 10000  # r in [3.57, 4.0] (chaotic regime)
    
    # Generate a salt value for additional security
    salt_part = int(key_hash[16:24], 16)
    salt = (salt_part % 10000) / 10000
    
    return x0, r, salt, key_hash[:16]  # Return short hash for display

# ====================== NORMAL ENCRYPTION ======================
def encrypt_image_normal(img, x0, r):
    img_arr = np.array(img, dtype=np.uint8)
    h, w, c = img_arr.shape
    total = h * w * c

    chaos = generate_logistic_sequence(x0, r, total)
    keystream = ((chaos * 1e6) % 256).astype(np.uint8)
    keystream = keystream.reshape(h, w, c)

    encrypted = np.bitwise_xor(img_arr, keystream)
    return Image.fromarray(encrypted), img_arr, encrypted

# ====================== ENCRYPTION WITH TARGET IMAGE ======================
def encrypt_image_with_target(original_img, target_img, x0, r):
    orig = np.array(original_img, dtype=np.uint8)
    target = np.array(target_img.resize(original_img.size), dtype=np.uint8)

    h, w, c = orig.shape
    total = h * w * c

    chaos = generate_logistic_sequence(x0, r, total)
    keystream = ((chaos * 1e6) % 256).astype(np.uint8)

    # FULL payload (TIDAK DISEMBUNYIKAN)
    payload = np.bitwise_xor(orig.flatten(), keystream)

    # Cipher = target murni (visual preserved)
    cipher_img = target.copy()

    return (
        Image.fromarray(cipher_img),
        payload,              # ← DISIMPAN
        orig.shape             # ← DISIMPAN
    )

# ====================== NORMAL DECRYPTION ======================
def decrypt_image_normal(encrypted_img, x0, r):
    enc_arr = np.array(encrypted_img, dtype=np.uint8)
    h, w, c = enc_arr.shape
    total = h * w * c

    chaos = generate_logistic_sequence(x0, r, total)
    keystream = ((chaos * 1e6) % 256).astype(np.uint8)
    keystream = keystream.reshape(h, w, c)

    decrypted = np.bitwise_xor(enc_arr, keystream)
    return Image.fromarray(decrypted), decrypted

# ====================== DECRYPTION WITH TARGET IMAGE ======================
def decrypt_image_with_target(payload, shape, x0, r):
    h, w, c = shape
    total = h * w * c

    chaos = generate_logistic_sequence(x0, r, total)
    keystream = ((chaos * 1e6) % 256).astype(np.uint8)

    recovered = np.bitwise_xor(payload, keystream)
    recovered_img = recovered.reshape(h, w, c)

    return Image.fromarray(recovered_img)


# ====================== ANALYTICAL FUNCTIONS ======================
def calculate_metrics(original, encrypted):
    """Calculate all encryption quality metrics."""
    
    if original.shape != encrypted.shape:
        raise ValueError("Image dimensions don't match!")
    
    if len(original.shape) == 3:
        orig_gray = np.mean(original, axis=2).astype(np.uint8)
        enc_gray = np.mean(encrypted, axis=2).astype(np.uint8)
    else:
        orig_gray = original
        enc_gray = encrypted
    
    # NPCR
    diff_pixels = np.sum(orig_gray != enc_gray)
    npcr = (diff_pixels / orig_gray.size) * 100
    
    # UACI
    uaci = np.mean(np.abs(orig_gray.astype(float) - enc_gray.astype(float))) / 255 * 100
    
    # Correlation
    h, w = orig_gray.shape
    num_samples = min(5000, h*w)
    indices = np.random.choice(h*w, num_samples, replace=False)
    orig_samples = orig_gray.flatten()[indices]
    enc_samples = enc_gray.flatten()[indices]
    correlation, _ = stats.pearsonr(orig_samples, enc_samples)
    
    # Entropy
    def calculate_entropy(channel):
        hist, _ = np.histogram(channel, bins=256, range=(0,256))
        prob = hist / hist.sum()
        prob = prob[prob > 0]
        return -np.sum(prob * np.log2(prob))
    
    if len(original.shape) == 3:
        entropies = [calculate_entropy(encrypted[:,:,i]) for i in range(3)]
        avg_entropy = np.mean(entropies)
    else:
        entropies = [calculate_entropy(encrypted)]
        avg_entropy = entropies[0]
    
    # PSNR and RMSE
    mse = np.mean((original.astype(float) - encrypted.astype(float)) ** 2)
    rmse = np.sqrt(mse)
    psnr = 20 * np.log10(255.0 / np.sqrt(mse)) if mse > 0 else float('inf')
    
    return {
        'npcr': npcr,
        'uaci': uaci,
        'correlation': correlation,
        'entropy': avg_entropy,
        'channel_entropies': entropies if len(original.shape)==3 else None,
        'psnr': psnr,
        'rmse': rmse,
        'mse': mse
    }

def display_analysis(metrics):
    """Display analysis results."""
    
    print("\n" + "="*70)
    print("📊 ENCRYPTION QUALITY ANALYSIS")
    print("="*70)
    
    print(f"\n🔢 QUANTITATIVE METRICS:")
    print(f"   • NPCR:       {metrics['npcr']:.6f}%")
    print(f"   • UACI:       {metrics['uaci']:.6f}%")
    print(f"   • Correlation: {metrics['correlation']:.6f}")
    print(f"   • Entropy:    {metrics['entropy']:.6f}")
    print(f"   • PSNR:       {metrics['psnr']:.2f} dB")
    print(f"   • RMSE:       {metrics['rmse']:.4f}")
    
    if metrics['channel_entropies']:
        print(f"   • R-Entropy:  {metrics['channel_entropies'][0]:.6f}")
        print(f"   • G-Entropy:  {metrics['channel_entropies'][1]:.6f}")
        print(f"   • B-Entropy:  {metrics['channel_entropies'][2]:.6f}")
    
    print(f"\n📈 INTERPRETATION:")
    
    if metrics['npcr'] > 99.6: npcr_verdict = "✅ EXCELLENT"
    elif metrics['npcr'] > 99.0: npcr_verdict = "✅ GOOD"
    elif metrics['npcr'] > 95.0: npcr_verdict = "⚠️  ACCEPTABLE"
    else: npcr_verdict = "❌ POOR"
    print(f"   • NPCR {npcr_verdict}: {metrics['npcr']:.4f}% (Ideal: >99.6%)")
    
    if 33.0 <= metrics['uaci'] <= 34.0: uaci_verdict = "✅ EXCELLENT"
    elif 32.0 <= metrics['uaci'] <= 35.0: uaci_verdict = "✅ GOOD"
    elif 30.0 <= metrics['uaci'] <= 37.0: uaci_verdict = "⚠️  ACCEPTABLE"
    else: uaci_verdict = "❌ POOR"
    print(f"   • UACI {uaci_verdict}: {metrics['uaci']:.4f}% (Ideal: 33.3-33.5%)")
    
    if abs(metrics['correlation']) < 0.01: corr_verdict = "✅ EXCELLENT"
    elif abs(metrics['correlation']) < 0.05: corr_verdict = "✅ GOOD"
    elif abs(metrics['correlation']) < 0.1: corr_verdict = "⚠️  ACCEPTABLE"
    else: corr_verdict = "❌ POOR"
    print(f"   • Correlation {corr_verdict}: {metrics['correlation']:.6f} (Ideal: 0)")
    
    if metrics['entropy'] > 7.997: entropy_verdict = "✅ EXCELLENT"
    elif metrics['entropy'] > 7.99: entropy_verdict = "✅ GOOD"
    elif metrics['entropy'] > 7.95: entropy_verdict = "⚠️  ACCEPTABLE"
    else: entropy_verdict = "❌ POOR"
    print(f"   • Entropy {entropy_verdict}: {metrics['entropy']:.6f} (Max: 8.0)")
    
    if metrics['psnr'] < 10: psnr_verdict = "✅ EXCELLENT"
    elif metrics['psnr'] < 20: psnr_verdict = "✅ GOOD"
    elif metrics['psnr'] < 30: psnr_verdict = "⚠️  ACCEPTABLE"
    else: psnr_verdict = "❌ POOR"
    print(f"   • PSNR {psnr_verdict}: {metrics['psnr']:.2f} dB (Good: <20 dB)")
    
    print("\n" + "="*70)

# ====================== SIMPLE FILE HANDLING FOR TUPLE FORMAT ======================
def get_uploaded_file_content(upload_widget):
    """Simple version that works with current ipywidgets format"""
    if not upload_widget.value:
        return None
    
    upload_data = upload_widget.value
    
    # Handle tuple format (what we have)
    if isinstance(upload_data, tuple) and len(upload_data) > 0:
        first_item = upload_data[0]
        
        # Try to get 'content' from dict/Bunch
        if hasattr(first_item, 'get'):
            content = first_item.get('content')
            
            # Convert memoryview/bytes to bytes
            if hasattr(content, 'tobytes'):
                return content.tobytes()
            elif isinstance(content, (bytes, bytearray, memoryview)):
                return bytes(content)
            else:
                return content
    
    return None

# ====================== USER INTERFACE ======================
# Create widgets
upload_original = widgets.FileUpload(accept='image/*', multiple=False, description='Original Image:')
upload_target = widgets.FileUpload(accept='image/*', multiple=False, description='Target Image:')

# Key input
key_input = widgets.Text(
    value='my_secret_key_123',
    placeholder='Enter encryption key',
    description='Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Parameter controls
r_slider = widgets.FloatSlider(
    value=3.9, min=3.57, max=4.0, step=0.01,
    description='r (growth rate):',
    style={'description_width': 'initial'}
)

encryption_mode = widgets.Dropdown(
    options=[
        ('Normal Encryption', 'normal'),
        ('Encrypt to Target Image', 'target')
    ],
    value='normal',
    description='Encryption Mode:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

blend_strength = widgets.FloatSlider(
    value=0.8, min=0.0, max=1.0, step=0.1,
    description='Encryption Strength:',
    style={'description_width': 'initial'}
)

# Buttons
encrypt_btn = widgets.Button(
    description='🔒 ENCRYPT IMAGE',
    button_style='warning',
    layout=widgets.Layout(width='200px', height='45px')
)

generate_key_btn = widgets.Button(
    description='🎲 Random Key',
    button_style='info',
    layout=widgets.Layout(width='150px', height='35px')
)

# Output area
output = widgets.Output()

# Parameter info display
param_info = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '10px'})

def generate_random_key(b):
    """Generate random key and update the input field"""
    random_key = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
    key_input.value = random_key
    update_parameter_info()
    with output:
        clear_output()
        print(f"✅ Random key generated: {random_key}")

def update_parameter_info():
    with param_info:
        clear_output()
        if key_input.value:
            try:
                x0, r_calculated, salt, key_hash = text_to_parameters(key_input.value, r_slider.value)
                print(f"🔑 Key Analysis:")
                print(f"   • Key: {key_input.value}")
                print(f"   • Hash: {key_hash}...")
                print(f"   • Generated x0: {x0:.6f}")
                print(f"   • Using r: {r_slider.value:.4f}")
                print(f"   • Chaotic Regime: {r_slider.value >= 3.57}")
                if encryption_mode.value == 'target':
                    print(f"   • Blend Strength: {blend_strength.value}")
            except Exception as e:
                print(f"⚠️  Error analyzing key: {e}")

# Initial update
update_parameter_info()

# Main processing function
def process_image(mode):
    with output:
        clear_output()

        if not upload_original.value:
            print("❌ ERROR: Please upload an original image first!")
            return

        if encryption_mode.value == 'target' and not upload_target.value:
            print("⚠️  WARNING: Target image not provided. Switching to normal encryption.")
            encryption_mode.value = 'normal'

        if not key_input.value:
            print("❌ ERROR: Please enter an encryption key!")
            return

        try:
            print("📥 Reading original image...")
            original_content = get_uploaded_file_content(upload_original)
            original_image = Image.open(io.BytesIO(original_content)).convert('RGB')
            print(f"✅ Original image loaded: {original_image.size}")

            # Generate parameters
            x0, r, salt, key_hash = text_to_parameters(key_input.value, r_slider.value)

            print(f"\n🚀 LOGISTIC MAP IMAGE ENCRYPTION")
            print("=" * 60)
            print(f"🔑 Key Hash: {key_hash}")
            print(f"• x0 = {x0:.6f}")
            print(f"• r  = {r:.4f}")
            print(f"• Mode = {encryption_mode.value.upper()}")

            if encryption_mode.value == 'target':
                print("\n🎯 ENCRYPTING TO TARGET IMAGE")
                target_content = get_uploaded_file_content(upload_target)
                target_image = Image.open(io.BytesIO(target_content)).convert('RGB')

                enc_img, payload, orig_shape = encrypt_image_with_target(
                    original_image, target_image, x0, r
                )

                orig_array = np.array(original_image)     # buat analisis
                enc_array  = np.array(enc_img)             # visual target

                dec_img = decrypt_image_with_target(
                    payload,
                    orig_shape,
                    x0,
                    r
                )

                dec_array = np.array(dec_img)

                fig, axes = plt.subplots(1, 4, figsize=(20, 5))
                axes[0].imshow(original_image)
                axes[0].set_title('Original')
                axes[1].imshow(target_image)
                axes[1].set_title('Target')
                axes[2].imshow(enc_img)
                axes[2].set_title('Encrypted (Target-like)')
                axes[3].imshow(dec_img)
                axes[3].set_title('Decrypted')
                for ax in axes: ax.axis('off')
                plt.show()


            else:
                print("\n🔒 NORMAL ENCRYPTION")
                enc_img, orig_array, enc_array = encrypt_image_normal(
                    original_image, x0, r
                )

                dec_img, dec_array = decrypt_image_normal(
                    enc_img, x0, r
                )

                fig, axes = plt.subplots(1, 4, figsize=(20, 5))
                axes[0].imshow(original_image); axes[0].set_title("Original")
                axes[1].text(0.5, 0.5, 'No Target\n(Normal Mode)',
                            ha='center', va='center', fontsize=12)
                axes[1].set_axis_off()
                axes[2].imshow(enc_img); axes[2].set_title("Encrypted (Noise)")
                axes[3].imshow(dec_img); axes[3].set_title("Decrypted (Recovered)")
                for ax in axes: ax.axis("off")
                plt.show()

            # Analysis
            print("\n📊 Performing encryption analysis...")
            metrics = calculate_metrics(orig_array, enc_array)
            display_analysis(metrics)

            print("\n✅ Encryption finished successfully!")

        except Exception as e:
            print(f"❌ ERROR: {e}")
            import traceback
            traceback.print_exc()

# Event handlers
def on_key_change(change):
    update_parameter_info()

def on_encryption_mode_change(change):
    with output:
        clear_output()
        if change['new'] == 'target':
            print("🎯 Target Image Mode Selected")
            print("   The encrypted image will blend with the target image.")
            print("   Upload a target image to use this feature.")
        else:
            print("🔒 Normal Encryption Mode Selected")
            print("   The encrypted image will look like random noise.")

def on_r_slider_change(change):
    update_parameter_info()

def on_blend_strength_change(change):
    update_parameter_info()

# Connect event handlers
key_input.observe(on_key_change, names='value')
encryption_mode.observe(on_encryption_mode_change, names='value')
r_slider.observe(on_r_slider_change, names='value')
blend_strength.observe(on_blend_strength_change, names='value')
generate_key_btn.on_click(generate_random_key)
encrypt_btn.on_click(lambda b: process_image('encrypt'))

# Display
print("🔐 ADVANCED LOGISTIC MAP IMAGE ENCRYPTION SYSTEM")
print("="*75)
print("Features:")
print("  1. Custom text key input")
print("  2. Encrypt to look like target image")
print("  3. Manual parameter adjustment")
print("  4. SHA-256 key hashing for security")

display(widgets.VBox([
    widgets.HTML("<h3 style='color:#2E86C1'>1. UPLOAD IMAGES</h3>"),
    widgets.HBox([upload_original, upload_target]),
    widgets.HTML("<h3 style='color:#2E86C1'>2. ENCRYPTION KEY</h3>"),
    widgets.HBox([key_input, generate_key_btn]),
    param_info,
    widgets.HTML("<h3 style='color:#2E86C1'>3. ENCRYPTION SETTINGS</h3>"),
    widgets.VBox([
        encryption_mode,
        widgets.HBox([r_slider]),
        widgets.HBox([blend_strength])
    ]),
    widgets.HTML("<h3 style='color:#2E86C1'>4. PROCESS IMAGES</h3>"),
    widgets.HBox([encrypt_btn],
                layout=widgets.Layout(justify_content='center', margin='20px 0')),
    widgets.HTML("<hr>"),
    output
]))

# Instructions
print("\nHOW TO USE:")
print("1. Upload your original image (required)")
print("2. Upload target image (for target mode)")
print("3. Enter an encryption key (or click Random Key)")
print("4. Choose encryption mode:")
print("   - Normal: Encrypts to noise")
print("   - Target: Encrypts to look like target image")
print("5. Adjust parameters if needed")
print("6. Click ENCRYPT or DECRYPT")
print("\nEXAMPLE (Target Mode):")
print("   Original: Flower image")
print("   Target: Frog image")
print("   Key: my_secret_password")
print("   Result: Encrypted image looks like frog, contains flower data")
print("   Decryption: With correct key, get back original flower")


🔐 ADVANCED LOGISTIC MAP IMAGE ENCRYPTION SYSTEM
Features:
  1. Custom text key input
  2. Encrypt to look like target image
  3. Manual parameter adjustment
  4. SHA-256 key hashing for security



HOW TO USE:
1. Upload your original image (required)
2. Upload target image (for target mode)
3. Enter an encryption key (or click Random Key)
4. Choose encryption mode:
   - Normal: Encrypts to noise
   - Target: Encrypts to look like target image
5. Adjust parameters if needed
6. Click ENCRYPT or DECRYPT

EXAMPLE (Target Mode):
   Original: Flower image
   Target: Frog image
   Key: my_secret_password
   Result: Encrypted image looks like frog, contains flower data
   Decryption: With correct key, get back original flower
